In [1]:
import json
import re
import pandas as pd

In [2]:
f = open("all_scripts_raw.json", "r")
data = json.load(f)

In [3]:
print(data.keys())
df = pd.DataFrame(data)
df.head()

dict_keys(['DS9', 'TOS', 'TAS', 'TNG', 'VOY', 'ENT'])


,DS9,TOS,TAS,TNG,VOY,ENT
episode 0,\n\n\n\n\n\nThe Deep Space Nine Transcripts - ...,\n\n\n\n\n\nThe Star Trek Transcripts - The Ca...,\n\n\n\n\n\nThe Animated Star Trek Transcripts...,\n\n\n\n\n\nThe Next Generation Transcripts - ...,\n\n\n\n\nThe Voyager Transcripts - Caretaker\...,\n\n\n\n\n\nThe Enterprise Transcripts - Broke...
episode 1,\n\n\n\n\n\nThe Deep Space Nine Transcripts - ...,\n\n\n\n\n\nThe Star Trek Transcripts - The Ma...,\n\n\n\n\n\nThe Animated Star Trek Transcripts...,\n\n\n\n\n\nThe Next Generation Transcripts - ...,\n\n\n\n\n\nThe Voyager Transcripts - Parallax...,\n\n\n\n\n\nThe Enterprise Transcripts - Fight...
episode 2,\n\n\n\n\n\nThe Deep Space Nine Transcripts - ...,\n\n\n\n\n\nThe Star Trek Transcripts - Charli...,\n\n\n\n\n\nThe Animated Star Trek Transcripts...,\n\n\n\n\n\nThe Next Generation Transcripts - ...,\n\n\n\n\n\nThe Voyager Transcripts - Time and...,\n\n\n\n\n\nThe Enterprise Transcripts - Stran...
episode 3,\n\n\n\n\n\nThe Deep Space Nine Transcripts - ...,\n\n\n\n\n\nThe Star Trek Transcripts - Where ...,\n\n\n\n\n\nThe Animated Star Trek Transcripts...,\n\n\n\n\n\nThe Next Generation Transcripts - ...,\n\n\n\n\n\n\nThe Voyager Transcripts - Phage\...,\n\n\n\n\n\nThe Enterprise Transcripts - Unexp...
episode 4,\n\n\n\n\n\nThe Deep Space Nine Transcripts - ...,\n\n\n\n\n\nThe Star Trek Transcripts - The Na...,\n\n\n\n\n\nThe Animated Star Trek Transcripts...,\n\n\n\n\n\nThe Next Generation Transcripts - ...,\n\n\n\n\n\nThe Voyager Transcripts - The Clou...,\n\n\n\n\n\nThe Enterprise Transcripts - Terra...


In [4]:
for series in df:
    for ep in df[series].index:
        if type(df[series][ep]) == str:
            df[series][ep] = re.sub(r'\s+', ' ', df[series][ep].strip())

# df['DS9'][0]

In [5]:
df_scenes = df.copy()

In [169]:
s = "This is a regex test [bracket] other text [Ops] is here [test] more text [OC] this is comms [on screen] yo"
scene_regex = re.compile(r'(?=[\[]+(?!OC)+(?!on))')
scene_regex.split(s)

['This is a regex test ',
 '[bracket] other text ',
 '[Ops] is here ',
 '[test] more text [OC] this is comms [on screen] yo']

In [170]:
for series in df_scenes:
    for ep in df_scenes[series].index:
        if type(df[series][ep]) == str:
            df_scenes[series][ep] = scene_regex.split(df_scenes[series][ep])

In [176]:
df_scenes['DS9'][1][:3]

['The Deep Space Nine Transcripts - Past Prologue Past Prologue Stardate: Unknown Original Airdate: 11 Jan, 1993 ',
 "[Replimat] GARAK: It's Doctor Bashir, isn't it? Of course it is. May I introduce myself? BASHIR: Er, yes, yes, of course. GARAK: My name is Garak. Cardassian by birth, obviously. The only one of us left on this station, as a matter of fact, so I do appreciate making new friends whenever I can. You are new to this station, I believe. BASHIR: I am, yes. Though, though I understand you've been here quite a while. GARAK: Ah, you know of me then. BASHIR: Would you care for some of this Tarkalean tea? It's very good. GARAK: What a thoughtful young man. How nice that we've met. BASHIR: You know, some people say that you remained on DS Nine as the eyes and ears of your fellow Cardassians. GARAK: You don't say? Doctor, you're not intimating that I'm considered some sort of spy, are you? BASHIR: I wouldn't know, sir. GARAK: Ah. An open mind. The essence of intellect. As you may a

In [186]:
re.split(r'[\s\[\]\(\),.?!:]+', df_scenes['DS9'][1][1].strip())

['',
 'Replimat',
 'GARAK',
 "It's",
 'Doctor',
 'Bashir',
 "isn't",
 'it',
 'Of',
 'course',
 'it',
 'is',
 'May',
 'I',
 'introduce',
 'myself',
 'BASHIR',
 'Er',
 'yes',
 'yes',
 'of',
 'course',
 'GARAK',
 'My',
 'name',
 'is',
 'Garak',
 'Cardassian',
 'by',
 'birth',
 'obviously',
 'The',
 'only',
 'one',
 'of',
 'us',
 'left',
 'on',
 'this',
 'station',
 'as',
 'a',
 'matter',
 'of',
 'fact',
 'so',
 'I',
 'do',
 'appreciate',
 'making',
 'new',
 'friends',
 'whenever',
 'I',
 'can',
 'You',
 'are',
 'new',
 'to',
 'this',
 'station',
 'I',
 'believe',
 'BASHIR',
 'I',
 'am',
 'yes',
 'Though',
 'though',
 'I',
 'understand',
 "you've",
 'been',
 'here',
 'quite',
 'a',
 'while',
 'GARAK',
 'Ah',
 'you',
 'know',
 'of',
 'me',
 'then',
 'BASHIR',
 'Would',
 'you',
 'care',
 'for',
 'some',
 'of',
 'this',
 'Tarkalean',
 'tea',
 "It's",
 'very',
 'good',
 'GARAK',
 'What',
 'a',
 'thoughtful',
 'young',
 'man',
 'How',
 'nice',
 'that',
 "we've",
 'met',
 'BASHIR',
 'You',
 'kno